# Brain Network Model
## Realistic structural connectivity + Jansen-Rit neural mass model

In [1]:
#import libs

import matplotlib
import numpy as np
import numpy
from tvb.simulator.lab import *
import multiprocessing as mp 
from tvb.datatypes.region_mapping import RegionMapping
from tvb.datatypes.projections import ProjectionMatrix, ProjectionSurfaceEEG
from tvb.datatypes.sensors import SensorsEEG
from tvb.datatypes import connectivity
from tvb.datatypes.cortex import Cortex
from tvb.datatypes import surfaces, local_connectivity, equations
from scipy import io as sio
from tvb.simulator.simulator import Simulator

In [ ]:
#initialize parameters varied in the grid search optimization, values are set now to oprimal value after optimization process
#structural parameters

global_coupling_strength = numpy.array([1.0]) #scale factor for global couplling interactions: grid search [0.01,0.1,1.0,10.0]
local_coupling_strength = numpy.array([0.5])  #scale factor for local couplling interactions: grid search [0.0001,0.001,0.01,0.1]
local_coupling_sigma = numpy.array([0.1])   #spatial spread (gaussian) for local couplling interactions: grid search[0.5,1.0,5.0]
conduction_velocity = numpy.array([25.0])  #scale factor for conduction velocity matrix: grid search [10.0,25.0,50.0,100.0]
#dynamical (jansen rit / noise) paramters
jrm.a = numpy.array([0.1])   #jansen rit excitatory time(inverse): grid search [0.01,0.1,1.0,10.0]
jrm.b = numpy.array([0.02])   #jansen rit inhibitory time (inverse): grid search [0.01,0.1,1.0,10.0]
jrm.mu = numpy.array([0.22])  #jansen rit mean firing rate background activity: grid search [0.12,0.22]
noise = numpy.array([10.0])  #scale factor for addictive noise in sde integration : grid search [1.0,10.0,100.0]

In [ ]:
#initialize structural global connectivity for both default and personalized connectome
#default connectivity. this can be use as a comparison or null model

con = connectivity.Connectivity.from_file('connectivity_76.zip') 
con.speed = np.array([10.0]) #constant velocity assumption
con.configure()

In [ ]:
#initialize personalized realistic connectivity from mri data

conn = connectivity.Connectivity.from_file(path_to_input_data+'sub-01_Connectome.zip')   #connectivity weigths HCPMMP1 atlas
conn.speed = numpy.loadtxt(path_to_input_data+'sub-01_conduction_velocities.txt', dtype=float) # heterogeneous conduction velocities
conn.speed[conn.speed==0]=0.0000000001 # avoid null problems
nr_streamlines = numpy.array([71631000]) # this is the nr of streamlines provided as input during Tractography
conn.weights = conn.weights/nr_streamlines
conn.configure()

In [ ]:
#initialize the Jansen-Rit neural mass model

jrm = models.JansenRit(a=jrm.a, b=jrm.b, v0=numpy.array([6.0]), nu_max=numpy.array([0.0025]), r=numpy.array([0.56]), J=numpy.array([135.0]), a_1=numpy.array([1.0]), a_2=numpy.array([0.8]), a_3=numpy.array([0.25]), a_4=numpy.array([0.25]), p_min=numpy.array([0.12]), p_max=numpy.array([0.32]), mu=jrm.mu,
                       variables_of_interest=("y1", "y2")
                      )

In [ ]:
#initialize the global coupling function (sigmoidal)

global_coupling_strength = numpy.array([1.0])   #this parameter must be set through the optimization process, running a grid search
global_coupling = coupling.SigmoidalJansenRit(cmin=numpy.array([0.0]), cmax=numpy.array([0.005]), midpoint=numpy.array([6.0]), r=numpy.array([0.56]), a=global_coupling_strength, dtype=float))
global_coupling.configure()

In [ ]:
#initialize the external background stimulation 

jrm.stvar = np.array([4])   # this applies the stimulation as additive term of the right part of the fourh equation of JansenRit (1995)
jrm.configure()
phi_n_scaling = phi_n_scaling_coeff * (jrm.a * jrm.A * (jrm.p_max-jrm.p_min) * 0.5 )**2 / 2.0
sigma         = numpy.zeros(6) 
sigma[4]      = phi_n_scaling # noise dispersion as drawn from a Gaussian distribution

In [ ]:
#initialize the scalp locations and the leadfieldamtrix for projection of cortical activity toward EEG electrodes

rm_f_name = (path_to_input_data+'sub-01_region_mapping.txt')
rm = RegionMapping.from_file(rm_f_name)
rm.connectivity=conn
sensorsEEG = SensorsEEG.from_file(path_to_input_data+'sub-01_EEG_Locations.txt')
sensorsEEG.configure()
prEEG = ProjectionSurfaceEEG.from_file(path_to_input_data+'sub-01_EEGProjection.mat', matlab_data_name="ProjectionMatrix")
prEEG.sensors=sensorsEEG

In [ ]:
#initialize the Heun stochastic integrator

my_seed = 42  # this is the default value in TVB
my_random_state = numpy.random.RandomState(int(my_seed))
fsamp = 1e3/256.0       # monitro sampling (256hz)
h=fsamp/8    # integration step aligned with eeg sampling frequency and to avoid numerical instability
my_noise=noise.Additive(nsig=sigma, ntau=0.0, noise_seed=my_seed, random_stream=my_random_state)
my_noise.configure()
heunint = integrators.HeunStochastic(
    dt=h, 
    noise=my_noise
)
heunint.configure()

In [ ]:
#initialize local coupling surface

ctx_surface_name=(path_to_input_data+'sub-01_Cortex.zip')
ctx_surface = surfaces.CorticalSurface.from_file(ctx_surface_name)
ctx_surface.zero_based_triangles=True
ctx_surface.configure()
prEEG.sources=ctx_surface
prEEG.configure()
rm.surface=ctx_surface
rm.configure()
loc_conn_matrix = sio.loadmat(path_to_input_data+local_connectivity_matrix_name)['local_connectivity_matrix']
loc_conn = local_connectivity.LocalConnectivity(surface=ctx_surface,
                                                matrix=loc_conn_matrix, equation=equations.Gaussian(), cutoff=local_connectivity_metadata['cutoff'])
loc_conn.equation.parameters['midpoint'] = 0.0
loc_conn.equation.parameters['offset'] = 0.0
loc_conn.equation.parameters['sigma'] = local_coupling_sigma 
loc_conn.configure()
ctx = Cortex.from_file(source_file=ctx_surface_name, region_mapping_file=rm_f_name, local_connectivity_file=None) 
ctx.surface.configure()
ctx.region_mapping_data.connectivity = conn
ctx.region_mapping_data.connectivity.configure()
ctx.region_mapping_data.configure()
ctx.local_connectivity = loc_conn
ctx.coupling_strength = local_coupling_strength
ctx.local_connectivity.configure()
ctx.configure()
init = my_random_state.rand(4000, int(jrm._nvar), int(ctx_surface.vertices.shape[0]+conn.cortical.shape[0]-numpy.count_nonzero(conn.cortical)), 1); 
set_surface_simulation=ctx

In [ ]:
#initialize EEG monitor. Extending to other monitors such as BOLD is possible within the same framework considering an appropriate hemodynamics function

mons = (
    monitors.EEG(sensors=sensorsEEG, projection=prEEG, region_mapping=rm, period=fsamp, reference=None, variables_of_interest=numpy.array([0,1])),
    monitors.Raw(),
    monitors.ProgressLogger(period=1.0),
)
mons[0].configure()
mons[1].configure()
mons[2].configure()

In [ ]:
#initialize the simulator

sim_length=22000.0 #[ms]
sim = Simulator(
    model=jrm,
    connectivity=conn,
    coupling=global_coupling,
    conduction_speed=conn_speed,
    integrator=heunint,
    monitors=mons,
    surface=set_surface_simulation,
    simulation_length=sim_length,
    initial_conditions=init
) 
sim.configure()

In [ ]:
#run the simulator

results = sim.run(simulation_length=sim_length,
                  random_state=my_random_state.get_state()
                  )

time_eeg = results[0][0]
eeg = results[0][1].squeeze() - results[1][1].squeeze()